# EJERCICIOS

In [1]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("pyspark_rdd").getOrCreate()
#spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

25/01/20 16:41:32 WARN Utils: Your hostname, iadb-12 resolves to a loopback address: 127.0.1.1; using 172.20.104.36 instead (on interface enp45s0)
25/01/20 16:41:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/20 16:41:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## EJERCICIO 0
En un documento word haz una lista de las diferentes operaciones con una breve descripción de lo que hace y un ejemplo de como se utiliza (justo la parte donde se utiliza sin contexto).

Ejemplo:

* map: aplica una funcion a todos los elementos de un rdd - rdd.map(lambda x: x * 2)

## EJERCICIO 1

Si tenemos dos RDD (A y B):
* rddA = sc.parallelize([1,2,3,4])
* rddB = sc.parallelize([3,4,5,6])

¿Cómo conseguimos los elementos que están en A y no B y los de B que no están en A? (es decir [1, 2, 5, 6]):

In [2]:
rddA = sc.parallelize([1,2,3,4])
rddB = sc.parallelize([3,4,5,6])

notA = rddB.subtract(rddA)
notB = rddA.subtract(rddB)
result = notB + notA

print(result.collect())

[1, 2, 5, 6]


## EJERCICIO 2

A partir de la lista implicita en el string "Perro Gato Loro Pez León Tortuga Gallina"
1.	Crea un RDD a partir de esta lista
2.	Convierte el RDD normal en un RDD de pares cuya clave sea la primera letra del animal
3.	Crea otro RDD de pares pero poniendo como clave un número incremental
4.	¿Y si queremos que el índice incremental empiece en 100?


In [ ]:
string = "Perro Gato Loro Pez León Tortuga Gallina"
lista = string.split(" ")
rdd = sc.parallelize(lista)

print(rdd.collect())

['Perro', 'Gato', 'Loro', 'Pez', 'León', 'Tortuga', 'Gallina']


## EJERCICIO 3

Dada la siguiente lista de compra:

lista = [('pan',3), ('agua',2), ('azúcar',1), ('leche',2), ('pan',1), ('cereales',3), ('agua',0.5), ('leche',2), ('filetes',5)]

Calcula:
1.	El total que se ha gastado por cada producto
2.	Cuánto es lo máximo que se ha pagado por cada producto

In [4]:
lista = [('pan',3), ('agua',2), ('azúcar',1), ('leche',2), ('pan',1), ('cereales',3), ('agua',0.5), ('leche',2), ('filetes',5)]

rdd = sc.parallelize(lista)
suma_cantidades = rdd.map(lambda x: x[1]).sum()
max_cantidad = rdd.map(lambda x: x[1]).max()

print(f"Suma de las cantidades: {suma_cantidades}")
print(f"Máxima cantidad: {max_cantidad}")

Suma de las cantidades: 19.5
Máxima cantidad: 5


## EJERCICIO 4

Ahora tenemos las cuentas de las compras de 3 días:
* día 1: pan 3€, agua 2€, azúcar 1€, leche 2€, pan 4€
* día 2: pan 1€, cereales 3€, agua 0.5€, leche 2€, filetes 5€
* día 3: filetes 2€, cereales 1€

Dada la siguiente lista de compra:

- dia1 = [('pan',3), ('agua',2), ('azúcar',1), ('leche',2), ('pan',4)]
- dia2 = [('pan',1), ('cereales',3), ('agua',0.5), ('leche',2), ('filetes',5)]
- dia3 = [('filetes',2), ('cereales',1)]

Responde:
1.	¿Cómo obtenemos lo que hemos gastado en cada producto?
2.	¿Y el gasto medio que hemos realizado en cada uno de ellos?

In [5]:
dia1 = [('pan',3), ('agua',2), ('azúcar',1), ('leche',2), ('pan',4)]
dia2 = [('pan',1), ('cereales',3), ('agua',0.5), ('leche',2), ('filetes',5)]
dia3 = [('filetes',2), ('cereales',1)]

rdd = sc.parallelize(dia1 + dia2 + dia3)
result = (
    rdd
    .groupByKey()
    .mapValues(lambda x: (sum(x), sum(x) / len(x)))
)

for producto, (total, promedio) in result.collect():
    print(f"Producto: {producto}, Total: {total}, Promedio: {promedio}")

Producto: pan, Total: 8, Promedio: 2.6666666666666665
Producto: azúcar, Total: 1, Promedio: 1.0
Producto: cereales, Total: 4, Promedio: 2.0
Producto: leche, Total: 4, Promedio: 2.0
Producto: agua, Total: 2.5, Promedio: 1.25
Producto: filetes, Total: 7, Promedio: 3.5


## EJERCICIO 5

A partir de la lista ciudades = ['Alicante', 'Elche', 'Valencia', 'Madrid', 'Barcelona', 'Bilbao', 'Sevilla'] crea rdds con las siguientes características:

1. Sólo las ciudades que tengan la letra e en su nombre y muéstralas.
2. Ciudades que tienen la letra e y el número de veces que aparece en cada nombre.
3. Averigua las ciudades que solo tengan una única e.
4. Nos han enviado una nueva lista pero no han separado bien las ciudades. Reorganiza la lista y colocalas correctamente, y cuenta las apariciones de la letra e de cada ciudad.

    ciudades_mal = [['Alicante.Elche', 'Valencia', 'Madrid.Barcelona', 'Bilbao.Sevilla'], ['Murcia', 'San Sebastián', 'Melilla.Aspe']]

In [6]:
ciudades = ['Alicante', 'Elche', 'Valencia', 'Madrid', 'Barcelona', 'Bilbao', 'Sevilla']

rdd = sc.parallelize(ciudades)
ciudades_con_e = rdd.filter(lambda ciudad: 'e' in ciudad.lower())

print(ciudades_con_e.collect())

['Alicante', 'Elche', 'Valencia', 'Barcelona', 'Sevilla']


In [7]:
ciudades_e_count = rdd.map(lambda ciudad: (ciudad, ciudad.lower().count('e'))).filter(lambda x: x[1] > 0)

print(ciudades_e_count.collect())

[('Alicante', 1), ('Elche', 2), ('Valencia', 1), ('Barcelona', 1), ('Sevilla', 1)]


In [8]:
ciudades_una_e = ciudades_e_count.filter(lambda x: x[1] == 1)

print(ciudades_una_e.collect())

[('Alicante', 1), ('Valencia', 1), ('Barcelona', 1), ('Sevilla', 1)]


In [9]:
ciudades_mal = [['Alicante.Elche', 'Valencia', 'Madrid.Barcelona', 'Bilbao.Sevilla'], 
                ['Murcia', 'San Sebastián', 'Melilla.Aspe']]

rdd_mal = sc.parallelize(ciudades_mal).flatMap(lambda sublist: sublist) # Fusiona listas
ciudades_organizadas = rdd_mal.flatMap(lambda x: x.split('.')) # Quita los '.'
ciudades_conteo_e = ciudades_organizadas.map(lambda ciudad: (ciudad, ciudad.lower().count('e')))

print(ciudades_conteo_e.collect())

[('Alicante', 1), ('Elche', 2), ('Valencia', 1), ('Madrid', 0), ('Barcelona', 1), ('Bilbao', 0), ('Sevilla', 1), ('Murcia', 0), ('San Sebastián', 1), ('Melilla', 1), ('Aspe', 1)]


## EJERCICIO 6

A partir de las siguientes listas:
* Inglés: hello, table, angel, cat, dog, animal, chocolate, dark, doctor, hospital, computer
* Español: hola, mesa, angel, gato, perro, animal, chocolate, oscuro, doctor, hospital, ordenador

Crea un RDD con tuplas de palabras y su traducción: 
[('hello', 'hola'),
 ('table', 'mesa'),
 ('angel', 'angel'),
 ('cat', 'gato')...]

Averigua:
1. Palabras que se escriben igual en inglés y en español
2. Palabras que en español son distintas que en inglés
3. Obtén una única lista con las palabras en ambos idiomas que son distintas entre ellas (['hello', 'hola', 'table', ...)
4. Haz dos grupos con todas las palabras, uno con las que empiezan por vocal y otro con las que empiecen por consonante.


In [10]:
ingles = ['hello', 'table', 'angel', 'cat', 'dog', 'animal', 'chocolate', 'dark', 'doctor', 'hospital', 'computer']
espanol = ['hola', 'mesa', 'angel', 'gato', 'perro', 'animal', 'chocolate', 'oscuro', 'doctor', 'hospital', 'ordenador']

rdd = sc.parallelize(zip(ingles, espanol))

print(rdd.collect())

[('hello', 'hola'), ('table', 'mesa'), ('angel', 'angel'), ('cat', 'gato'), ('dog', 'perro'), ('animal', 'animal'), ('chocolate', 'chocolate'), ('dark', 'oscuro'), ('doctor', 'doctor'), ('hospital', 'hospital'), ('computer', 'ordenador')]


In [11]:
iguales = rdd.filter(lambda x: x[0] == x[1])

print(iguales.collect())

[('angel', 'angel'), ('animal', 'animal'), ('chocolate', 'chocolate'), ('doctor', 'doctor'), ('hospital', 'hospital')]


In [12]:
diferentes = rdd.filter(lambda x: x[0] != x[1])

print(diferentes.collect())

[('hello', 'hola'), ('table', 'mesa'), ('cat', 'gato'), ('dog', 'perro'), ('dark', 'oscuro'), ('computer', 'ordenador')]


In [13]:
lista_distintas = diferentes.flatMap(lambda x: x).distinct()

print(lista_distintas.collect())

['perro', 'hola', 'ordenador', 'hello', 'mesa', 'gato', 'oscuro', 'cat', 'dog', 'computer', 'dark', 'table']


In [14]:
all = rdd.flatMap(lambda x: x)

vocales = all.filter(lambda palabra: palabra[0].lower() in 'aeiou')
consonantes = all.filter(lambda palabra: palabra[0].lower() not in 'aeiou')

# Mostrar los grupos
print("Palabras que empiezan por vocal:", vocales.collect())
print("Palabras que empiezan por consonante:", consonantes.collect())

Palabras que empiezan por vocal: ['angel', 'angel', 'animal', 'animal', 'oscuro', 'ordenador']
Palabras que empiezan por consonante: ['hello', 'hola', 'table', 'mesa', 'cat', 'gato', 'dog', 'perro', 'chocolate', 'chocolate', 'dark', 'doctor', 'doctor', 'hospital', 'hospital', 'computer']


## EJERCICIO 7

A partir del fichero de El Quijote:
1. Crear un rdd con todas las palabras del documento.
2. ¿Cuantas veces aparece la palabra Dulcinea (independientemente de si está en mayúsculas o minúsculas)? ¿Y Rocinante?
3. Devuelve una lista ordenada según el número de veces que sale cada palabra de más a menos.
4. Guarda el resultado en el disco

In [15]:
rdd_texto = sc.textFile('Datos/el_quijote.txt')
rdd_palabras = rdd_texto.flatMap(lambda linea: linea.split())

print(rdd_palabras.take(10))
print(rdd_palabras.count())

['The', 'Project', 'Gutenberg', 'eBook', 'of', 'Don', 'Quijote', 'This', 'ebook', 'is']
389651


In [16]:
rdd_palabras_normalizadas = rdd_palabras.map(lambda palabra: palabra.lower())
dulcinea_count = rdd_palabras_normalizadas.filter(lambda palabra: palabra == 'dulcinea').count()
rocinante_count = rdd_palabras_normalizadas.filter(lambda palabra: palabra == 'rocinante').count()

print(f"Dulcinea: {dulcinea_count} veces")
print(f"Rocinante: {rocinante_count} veces")

Dulcinea: 171 veces
Rocinante: 74 veces


In [18]:
conteo_palabras = rdd_palabras_normalizadas.map(lambda palabra: (palabra, 1)).reduceByKey(lambda a, b: a + b)
palabras_ordenadas = conteo_palabras.sortBy(lambda x: x[1], ascending=False)

print(palabras_ordenadas.take(10))
palabras_ordenadas.saveAsTextFile('Datos/el_quijote_pycount')

[('que', 19627), ('de', 18384), ('y', 16646), ('la', 10486), ('a', 9899), ('el', 8254), ('en', 8240), ('no', 6034), ('los', 4764), ('se', 4751)]


## EJERCICIO 8

Dada una cadena que contiene una lista de nombres Juan, Jimena, Luis, Cristian, Laura, Lorena, Cristina, Jacobo, Jorge:
1. Tranforma la cadena en una lista y luego en un RDD
2. Agrúpalos según su inicial, de manera que tengamos tuplas formadas por la letra inicial y todos los nombres que comienzan por dicha letra:

    [('J', ['Juan', 'Jimena', 'Jacobo', 'Jorge']),

    ('L', ['Luis', 'Laura', 'Lorena']),
    
    ('C', ['Cristian', 'Cristina'])]
3. De la lista original, obtén una muestra de 5 elementos sin repetir valores.
4. Devuelve una muestra de datos de aproximadamente la mitad de registros que la lista original con datos que pudieran llegar a repetirse.

In [2]:
nombres = "Juan, Jimena, Luis, Cristian, Laura, Lorena, Cristina, Jacobo, Jorge"
lista = nombres.split(", ")
rdd = sc.parallelize(lista)

print(rdd.collect())

['Juan', 'Jimena', 'Luis', 'Cristian', 'Laura', 'Lorena', 'Cristina', 'Jacobo', 'Jorge']


In [3]:
rdd_agrupado = rdd.map(lambda nombre: (nombre[0], nombre)) \
                           .groupByKey() \
                           .map(lambda x: (x[0], list(x[1])))

print(rdd_agrupado.collect())

25/01/20 16:41:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


[('L', ['Luis', 'Laura', 'Lorena']), ('C', ['Cristian', 'Cristina']), ('J', ['Juan', 'Jimena', 'Jacobo', 'Jorge'])]


In [4]:
muestra_sin_repetir = rdd.takeSample(False, 5)

print(muestra_sin_repetir)

['Jimena', 'Jorge', 'Luis', 'Cristina', 'Cristian']


In [7]:
muestra_con_repetir = rdd.sample(True, 0.5)

print(muestra_con_repetir.collect())

['Juan', 'Juan', 'Jimena', 'Cristian', 'Cristian', 'Laura']


## EJERCICIO 9

Dada una lista de elementos desordenados y algunos repetidos, mediante rdd devolver una muestra de 5 elementos, que estén en la lista, sin repetir y ordenados descendentemente.

lista = [4,6,34,7,9,2,3,4,4,21,4,6,8,9,7,8,5,4,3,22,34,56,98]

1. Selecciona el elemento mayor de la lista resultante.
2. Muestra los dos elementos menores.

In [8]:
lista = [4,6,34,7,9,2,3,4,4,21,4,6,8,9,7,8,5,4,3,22,34,56,98]

rdd = sc.parallelize(lista)
muestra = rdd.distinct().takeOrdered(5, key=lambda x: -x)

print("Muestra de 5 elementos únicos y ordenados descendentemente:", muestra)

Muestra de 5 elementos únicos y ordenados descendentemente: [98, 56, 34, 22, 21]


In [9]:
menores = sorted(muestra)[:2]
print("Los dos elementos menores de la muestra:", menores)

Los dos elementos menores de la muestra: [21, 22]


## EJERCICIO 10

En una red social sobre cine, tenemos un fichero ratings.txt compuesta por el código de la película, el código del usuario, la calificación asignada y el TIMESTAMP de la votación con el siguiente formato:

1::1193::5::978300760

1::661::3::978302109

1::914::3::978301968

Se pide:
1. Obtener para cada película, la nota media de todas sus votaciones.
2. Películas cuya nota media sea superior a 3.

## EJERCICIO 11

Tenemos las calificaciones de las asignaturas de matemáticas (notas_mates.txt), inglés (notas_ingles.txt) y física (notas_fisica.txt) de los alumnos del instituto en 3 documentos de texto. A partir de estos ficheros:
1. Crea 3 RDD de pares, uno para cada asignatura, con los alumnos y sus notas
2. Crea un solo RDD con todas las notas (cada entrada es un usuario con sus tres notas)
3. ¿Cuál es la nota más baja que ha tenido cada alumno?
4. ¿Cuál es la nota media de cada alumno?
5. ¿Cuántos estudiantes suspende cada asignatura?
         [('Matemáticas', 7), ('Física', 8), ('Inglés', 7)]
6. ¿En qué asignatura suspende más gente?
7. Total de notables o sobresalientes por alumno, es decir, cantidad de notas superiores o igual a 7.
8. ¿Qué alumno no se ha presentado a inglés?
9. ¿A cuántas asignaturas se ha presentado cada alumno?

## EJERCICIO 12

Realiza las siguientes operaciones:
1. Genera un RDD con la siguiente lista [1, 2, 3, 3, 4, 5, 6]
2. Multiplica por 2 todos los elementos del RDD anterior
3. Filtra el RDD inicial por los elementos pares
4. Muestra los elementos unicos del RDD inicial
5. Toma los elementos del RDD inicial para multiplicarlos entre si y obtener un resultado

## EJERCICIO 13

rdd = sc.parallelize([('a', 1), ('a', 2), ('a', 3), ('b', 1)])

Realiza las siguientes operaciones:
1. Obten la suma de los valores agrupados por el key
2. Ordena el RDD inicial en base al key, de forma descendente

## EJERCICIO 14

data= ['Python', 'Scala', 'Python', 'R', 'Python', 'Java', 'R' ]

rdd= sc.parallelize(data)

Cuenta cuantas veces aparece cada valor